In [450]:
# importing some necessary libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from importlib import reload
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

For my FCA analysis, I use tic-tac-toe dataset. It consists of 9 variables and 1 target value. I already have train and test dataframes

In [24]:
df_train = pd.read_csv('train1.csv'.format(10))
df_train.head()


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,o,b,b,positive
3,x,x,x,x,o,o,b,o,b,positive
4,x,x,x,x,o,o,b,b,o,positive


In [25]:
df_train["V10"]



0      positive
1      positive
2      positive
3      positive
4      positive
         ...   
860    negative
861    negative
862    negative
863    negative
864    negative
Name: V10, Length: 865, dtype: object

In [26]:
df_test = pd.read_csv('test1.csv'.format(10))
df_test.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,b,o,b,o,positive
2,x,x,x,o,o,x,o,x,o,positive
3,x,x,x,o,o,b,x,o,b,positive
4,x,x,x,b,o,b,o,o,x,positive


In [27]:
df_train.shape

(865, 10)

First, we need to scale out data. I want to replace x in columns with 1 and o with 0.

In [28]:
df_train = df_train.applymap(lambda el: 1 if el in ["x", "positive"] else 0)
df_test = df_test.applymap(lambda el: 1 if el in ["x", "positive"] else 0)



In [29]:
df_train.head()



,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,0,1,1
2,1,1,1,1,0,0,0,0,0,1
3,1,1,1,1,0,0,0,0,0,1
4,1,1,1,1,0,0,0,0,0,1


In [30]:
df_test.head()




,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,1,1,1,1,0,0,0,1,0,1
1,1,1,1,1,0,0,0,0,0,1
2,1,1,1,0,0,1,0,1,0,1
3,1,1,1,0,0,0,1,0,0,1
4,1,1,1,0,0,0,0,0,1,1


## FCA

In [442]:
def lazy_fca_alg0(train_dt, test_dt):
    

    
    ftrs = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Sex',
       'Jaundice', 'Family_mem_with_ASD', 'Age_Mons_(11.999, 24.0]',
       'Age_Mons_(24.0, 34.0]', 'Age_Mons_(34.0, 36.0]']
    
    def intersect(example):
        return set([i+':'+str(j) for i, j in zip(ftrs, example)])

    plus=[]
    minus=[]
    x_test=[]
    y_test=[]
    
    q = open(train_dt + ".csv", "r")
    positive = "positive"
    negative = "negative"
    train = [a.strip().split(",") for a in q]
    _plus = [intersect(a[:-1]) for a in train if a[-1] == positive]
    _minus = [intersect(a[:-1]) for a in train if a[-1] == negative]
    q.close()
    plus+=_plus
    minus+=_minus  
    
    w = open(test_dt + ".csv", "r")
    _unknown = [a.strip().split(",") for a in w]

    _x_test = [intersect(a[:-1]) for a in _unknown]
    _y_test = [1 if a[-1] == positive else 0 for a in _unknown ]
    del _x_test[0]
    del _y_test[0]
    w.close()
    x_test+=_x_test
    y_test+=_y_test
    
    y_pred=[]
    for i in x_test:
        unkn_set=i
        pos=0
        neg=0

        for j in plus:
            pos_set=j
            res=pos_set & unkn_set
            pos+=float(len(res)) / len(pos_set)
        pos=float(pos) 

        for j in minus:
            neg_set=j
            res=neg_set & unkn_set
            neg+=float(len(res)) / len(neg_set)
        neg=float(neg) 

        if (pos < neg):
            y_pred.append(0)
        else:
            y_pred.append(1)

    y_pred=np.array(y_pred)
    y_test=np.array(y_test)

    TP = np.sum(y_test * y_pred)
    TN = np.sum(y_test + y_pred == 0)
    FP = np.sum((y_test  == 0) * (y_pred == 1))
    FN = np.sum((y_test  == 1) * (y_pred == 0))
    try:
        TPR = float(TP) / (TP + FN)
        TNR = float(TN) / (TN + FP)
        FPR = float(FP) / (FP + TN)
        NPV = float(TN) / (TN + FN)
        FDR = float(FP) / (TP + FP)
        acc = (TP+TN)/(TP + FP + TN + FN)
        prec = TP/(TP + FP)
        rec = TP/(TP + FN)
        print('''
True Positive: {}
True Negative: {} 
False Positive: {}
False Negative: {}
True Positive Rate: {}
True Negative Rate: {}
Negative Predictive Value: {}
False Positive Rate: {}
False Discovery Rate: {}
Accuracy: {}
Precision: {}
Recall: {}'''.format(TP, TN, FP, FN, TPR, TNR, NPV, FPR, FDR,acc, prec, rec))
    except:
        print('sorry')

In [443]:
lazy_fca_alg0("train1", "test1")


True Positive: 61
True Negative: 0 
False Positive: 32
False Negative: 0
True Positive Rate: 1.0
True Negative Rate: 0.0
Negative Predictive Value: nan
False Positive Rate: 1.0
False Discovery Rate: 0.34408602150537637
Accuracy: 0.6559139784946236
Precision: 0.6559139784946236
Recall: 1.0


<ipython-input-442-1709ed341262>:72: RuntimeWarning: invalid value encountered in true_divide
  NPV = float(TN) / (TN + FN)


In [435]:
def lazy_fca_alg1(train_dt, test_dt):
    

    
    ftrs = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Sex',
       'Jaundice', 'Family_mem_with_ASD', 'Age_Mons_(11.999, 24.0]',
       'Age_Mons_(24.0, 34.0]', 'Age_Mons_(34.0, 36.0]']
    
    def intersect(example):
        return set([i+':'+str(j) for i, j in zip(ftrs, example)])

    plus=[]
    minus=[]
    x_test=[]
    y_test=[]
    
    q = open(train_dt + ".csv", "r")
    positive = "positive"
    negative = "negative"
    train = [a.strip().split(",") for a in q]
    _plus = [intersect(a[:-1]) for a in train if a[-1] == positive]
    _minus = [intersect(a[:-1]) for a in train if a[-1] == negative]
    q.close()
    plus+=_plus
    minus+=_minus  
    
    w = open(test_dt + ".csv", "r")
    _unknown = [a.strip().split(",") for a in w]

    _x_test = [intersect(a[:-1]) for a in _unknown]
    _y_test = [1 if a[-1] == positive else 0 for a in _unknown ]
    del _x_test[0]
    del _y_test[0]
    w.close()
    x_test+=_x_test
    y_test+=_y_test
    
    y_pred=[]
    for i in x_test:
        unkn_set=i
        pos=0
        neg=0

        for j in plus:
            pos_set=j
            res=pos_set & unkn_set
            pos+=float(len(res)) / len(pos_set)
        pos=float(pos) / len(plus) 

        for j in minus:
            neg_set=j
            res=neg_set & unkn_set
            neg+=float(len(res)) / len(neg_set)
        neg=float(neg) / len(minus) 

        if (pos < neg):
            y_pred.append(0)
        else:
            y_pred.append(1)

    y_pred=np.array(y_pred)
    y_test=np.array(y_test)

    TP = np.sum(y_test * y_pred)
    TN = np.sum(y_test + y_pred == 0)
    FP = np.sum((y_test  == 0) * (y_pred == 1))
    FN = np.sum((y_test  == 1) * (y_pred == 0))
    try:
        TPR = float(TP) / (TP + FN)
        TNR = float(TN) / (TN + FP)
        FPR = float(FP) / (FP + TN)
        NPV = float(TN) / (TN + FN)
        FDR = float(FP) / (TP + FP)
        acc = (TP+TN)/(TP + FP + TN + FN)
        prec = TP/(TP + FP)
        rec = TP/(TP + FN)
        print('''
True Positive: {}
True Negative: {} 
False Positive: {}
False Negative: {}
True Positive Rate: {}
True Negative Rate: {}
Negative Predictive Value: {}
False Positive Rate: {}
False Discovery Rate: {}
Accuracy: {}
Precision: {}
Recall: {}'''.format(TP, TN, FP, FN, TPR, TNR, NPV, FPR, FDR,acc, prec, rec))
    except:
        print('sorry')

In [437]:
lazy_fca_alg1("train1", "test1")


True Positive: 34
True Negative: 24 
False Positive: 8
False Negative: 27
True Positive Rate: 0.5573770491803278
True Negative Rate: 0.75
Negative Predictive Value: 0.47058823529411764
False Positive Rate: 0.25
False Discovery Rate: 0.19047619047619047
Accuracy: 0.6236559139784946
Precision: 0.8095238095238095
Recall: 0.5573770491803278


In [440]:
def lazy_fca_alg2(train_dt, test_dt):
    

    
    ftrs = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Sex',
       'Jaundice', 'Family_mem_with_ASD', 'Age_Mons_(11.999, 24.0]',
       'Age_Mons_(24.0, 34.0]', 'Age_Mons_(34.0, 36.0]']
    
    def intersect(example):
        return set([i+':'+str(j) for i, j in zip(ftrs, example)])

    plus=[]
    minus=[]
    x_test=[]
    y_test=[]
    
    q = open(train_dt + ".csv", "r")
    positive = "positive"
    negative = "negative"
    train = [a.strip().split(",") for a in q]
    _plus = [intersect(a[:-1]) for a in train if a[-1] == positive]
    _minus = [intersect(a[:-1]) for a in train if a[-1] == negative]
    q.close()
    plus+=_plus
    minus+=_minus  
    
    w = open(test_dt + ".csv", "r")
    _unknown = [a.strip().split(",") for a in w]

    _x_test = [intersect(a[:-1]) for a in _unknown]
    _y_test = [1 if a[-1] == positive else 0 for a in _unknown ]
    del _x_test[0]
    del _y_test[0]
    w.close()
    x_test+=_x_test
    y_test+=_y_test
    
    y_pred=[]
    counter=0
    for i in x_test:
        counter+=1
        unkn_set=i
        pos=0
        neg=0

        for j in plus:
            pos_set=j
            res=pos_set & unkn_set
            if len(res)!=0:
                for k in plus:
                    if k.issuperset(res) and k!=j:
                        #pos+= 1.0 * len(res) / len(plus) #float(len(k&res)) / len(res) * float(len(res)) / len (pos_set) / len(plus)
                        pos+= float(len(k&res)) / len(res) * float(len(res)) / len (pos_set) / len(plus)
                        
        pos=  float(pos) / len(plus) 

        for j in minus:
            neg_set=j
            res=neg_set & unkn_set
            if len(res)!=0:
                for k in minus :
                    if k.issuperset(res) and k!=j:
                        #neg+= 1.0 * len(res) / len(minus) #float(len(k&res)) / len(res) * float(len(res)) / len(neg_set) / len(minus)
                        neg+= float(len(k&res)) / len(res) * float(len(res)) / len(neg_set) / len(minus)

        neg =  float(neg) / len(minus) 

        if (pos < neg):
            y_pred.append(0)
        else:
            if (pos==0 and neg==0):
                print('i dont know')
                y_pred.append(0.5)
            else:
                y_pred.append(1)

    y_pred=np.array(y_pred)
    y_test=np.array(y_test)

    TP = np.sum(y_test * y_pred)
    TN = np.sum(y_test + y_pred == 0)
    FP = np.sum((y_test  == 0) * (y_pred == 1))
    FN = np.sum((y_test  == 1) * (y_pred == 0))
    try:
        TPR = float(TP) / (TP + FN)
        TNR = float(TN) / (TN + FP)
        FPR = float(FP) / (FP + TN)
        NPV = float(TN) / (TN + FN)
        FDR = float(FP) / (TP + FP)
        acc = (TP+TN)/(TP + FP + TN + FN)
        prec = TP/(TP + FP)
        rec = TP/(TP + FN)
        print('''
True Positive: {}
True Negative: {} 
False Positive: {}
False Negative: {}
True Positive Rate: {}
True Negative Rate: {}
Negative Predictive Value: {}
False Positive Rate: {}
False Discovery Rate: {}
Accuracy: {}
Precision: {}
Recall: {}'''.format(TP, TN, FP, FN, TPR, TNR, NPV, FPR, FDR,acc, prec, rec))
    except:
        print('sorry')

In [441]:
lazy_fca_alg2("train1", "test1")


True Positive: 55
True Negative: 32 
False Positive: 0
False Negative: 6
True Positive Rate: 0.9016393442622951
True Negative Rate: 1.0
Negative Predictive Value: 0.8421052631578947
False Positive Rate: 0.0
False Discovery Rate: 0.0
Accuracy: 0.9354838709677419
Precision: 1.0
Recall: 0.9016393442622951


## Comparing with KNN algorithm (state of the art algorithm)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
def knn_alg(df_train, df_test):
    
    clf = KNeighborsClassifier(n_neighbors=5)
    clf = clf.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1])
     
    df_train = df_train.to_numpy()
    df_test = df_test.to_numpy()
    
# our metrics
    tp = 0
    fp = 0    
    tn = 0
    fn = 0
    
    tp_list = []
    fp_list = []
    tn_list = []
    fn_list = []
    


    
# spliting data into two parts: C+ - context with "+" examples, C- - context with "-" examples:

    plus_context_train = df_train[df_train[:, -1] == 1][:, :-1] 
    minus_context_train = df_train[df_train[:, -1] == 0][:, :-1]
    plus_context_test = df_test[df_test[:, -1] == 1][:, :-1] 
    minus_context_test = df_test[df_test[:, -1] == 0][:, :-1]
    
    TP= int(clf.predict(plus_context_test).sum())
    FN=(1 - clf.predict(plus_context_test)).sum()
    TN=(1 - clf.predict(minus_context_test)).sum()
    FP= int(clf.predict(minus_context_test).sum())
    
    print (
        "True positive:", TP ,"\n" 
        "True Negative:", TN,"\n"
        "False Positive:", FP,"\n"
        "False Negative:", FN,"\n"
        "True Positive Rate:", TP/(TP + FN),"\n"
        "True Negative Rate:", TN/(TN + FP),"\n"
        "Negative Predictive Value:", TN/(TN + FN),"\n"
        "False Positive Rate:",FP/(FP + TN),"\n"
        "False Discovery Rate:",FP/(TP + FP),"\n"
        "Accuracy:" ,(TP+TN)/(TP + FP + TN + FN),"\n"
        "Precision:" ,TP/(TP + FP),"\n"
        "Recall:" ,TP/(TP + FN))
    
    

In [12]:
knn_alg(df_train, df_test)

True positive: 60 
True Negative: 26 
False Positive: 6 
False Negative: 1 
True Positive Rate: 0.9836065573770492 
True Negative Rate: 0.8125 
Negative Predictive Value: 0.9629629629629629 
False Positive Rate: 0.1875 
False Discovery Rate: 0.09090909090909091 
Accuracy: 0.9247311827956989 
Precision: 0.9090909090909091 
Recall: 0.9836065573770492


In [460]:
from sklearn.linear_model import LogisticRegression
def log_reg(df_train, df_test):
    
    clf = LogisticRegression(random_state=2)
    clf = clf.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1])
     
    df_train = df_train.to_numpy()
    df_test = df_test.to_numpy()
    
# our metrics
    tp = 0
    fp = 0    
    tn = 0
    fn = 0
    
    tp_list = []
    fp_list = []
    tn_list = []
    fn_list = []
    


    
# spliting data into two parts: C+ - context with "+" examples, C- - context with "-" examples:

    plus_context_train = df_train[df_train[:, -1] == 1][:, :-1] 
    minus_context_train = df_train[df_train[:, -1] == 0][:, :-1]
    plus_context_test = df_test[df_test[:, -1] == 1][:, :-1] 
    minus_context_test = df_test[df_test[:, -1] == 0][:, :-1]
    
    TP= int(clf.predict(plus_context_test).sum())
    FN=(1 - clf.predict(plus_context_test)).sum()
    TN=(1 - clf.predict(minus_context_test)).sum()
    FP= int(clf.predict(minus_context_test).sum())
    
    print (
        "True positive:", TP ,"\n" 
        "True Negative:", TN,"\n"
        "False Positive:", FP,"\n"
        "False Negative:", FN,"\n"
        "True Positive Rate:", TP/(TP + FN),"\n"
        "True Negative Rate:", TN/(TN + FP),"\n"
        "Negative Predictive Value:", TN/(TN + FN),"\n"
        "False Positive Rate:",FP/(FP + TN),"\n"
        "False Discovery Rate:",FP/(TP + FP),"\n"
        "Accuracy:" ,(TP+TN)/(TP + FP + TN + FN),"\n"
        "Precision:" ,TP/(TP + FP),"\n"
        "Recall:" ,TP/(TP + FN))
    
    

In [461]:
log_reg(df_train, df_test)

True positive: 53 
True Negative: 15 
False Positive: 17 
False Negative: 8 
True Positive Rate: 0.8688524590163934 
True Negative Rate: 0.46875 
Negative Predictive Value: 0.6521739130434783 
False Positive Rate: 0.53125 
False Discovery Rate: 0.24285714285714285 
Accuracy: 0.7311827956989247 
Precision: 0.7571428571428571 
Recall: 0.8688524590163934


As we see k-nearest neighbors algorithm showed much better results

## Autism dataset

In [400]:
auitsm_df = pd.read_csv("autism.csv")
auitsm_df.head()


,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


In [401]:
auitsm_df = auitsm_df.dropna()


In [402]:
auitsm_df['Age_Mons'] = pd.qcut(auitsm_df['Age_Mons'], 3)

Sex = {'m': 1,'f': 0}
auitsm_df.Sex = [Sex[item] for item in auitsm_df.Sex] 

Jaundice = {'yes': 1,'no': 0}
auitsm_df.Jaundice = [Jaundice[item] for item in auitsm_df.Jaundice] 

Family_mem_with_ASD = {'yes': 1,'no': 0}
auitsm_df.Family_mem_with_ASD = [Family_mem_with_ASD[item] for item in auitsm_df.Family_mem_with_ASD] 

Class = {'Yes': 'positive','No': 'negative'}
auitsm_df.Class = [Class[item] for item in auitsm_df.Class] 

auitsm_df = pd.get_dummies(auitsm_df, columns = ['Age_Mons'])



In [403]:
auitsm_df = auitsm_df.drop(["Case_No", "Qchat-10-Score", "Ethnicity", "Who completed the test"], axis = 1)
auitsm_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sex,Jaundice,Family_mem_with_ASD,Class,"Age_Mons_(11.999, 24.0]","Age_Mons_(24.0, 34.0]","Age_Mons_(34.0, 36.0]"
0,0,0,0,0,0,0,1,1,0,1,0,1,0,negative,0,1,0
1,1,1,0,0,0,1,1,0,0,0,1,1,0,positive,0,0,1
2,1,0,0,0,0,0,1,1,0,1,1,1,0,positive,0,0,1
3,1,1,1,1,1,1,1,1,1,1,1,0,0,positive,1,0,0
4,1,1,0,1,1,1,1,1,1,1,0,0,1,positive,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,0,0,0,0,0,0,0,0,0,1,0,0,1,negative,1,0,0
1050,0,0,1,1,1,0,1,0,1,0,1,1,0,positive,1,0,0
1051,1,0,1,1,1,1,1,1,1,1,1,1,0,positive,1,0,0
1052,1,0,0,0,0,0,0,1,0,1,1,0,1,negative,1,0,0


In [404]:
auitsm_df = auitsm_df[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Sex',
       'Jaundice', 'Family_mem_with_ASD', 'Age_Mons_(11.999, 24.0]',
       'Age_Mons_(24.0, 34.0]', 'Age_Mons_(34.0, 36.0]','Class']]

In [405]:
auitsm_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sex,Jaundice,Family_mem_with_ASD,"Age_Mons_(11.999, 24.0]","Age_Mons_(24.0, 34.0]","Age_Mons_(34.0, 36.0]",Class
0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,negative
1,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,positive
2,1,0,0,0,0,0,1,1,0,1,1,1,0,0,0,1,positive
3,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,positive
4,1,1,0,1,1,1,1,1,1,1,0,0,1,1,0,0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,negative
1050,0,0,1,1,1,0,1,0,1,0,1,1,0,1,0,0,positive
1051,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,0,positive
1052,1,0,0,0,0,0,0,1,0,1,1,0,1,1,0,0,negative


In [406]:
from sklearn.model_selection import train_test_split

auitsm_df_train, auitsm_df_test = train_test_split(auitsm_df, test_size=0.5)

In [407]:
auitsm_df_train.to_csv('auitsm_df_train.csv',index=False)
auitsm_df_test.to_csv('auitsm_df_test.csv',index=False)

In [408]:
auitsm_df_train

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sex,Jaundice,Family_mem_with_ASD,"Age_Mons_(11.999, 24.0]","Age_Mons_(24.0, 34.0]","Age_Mons_(34.0, 36.0]",Class
163,0,0,0,0,1,1,1,0,0,0,1,0,1,1,0,0,negative
241,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,positive
198,1,0,1,1,1,1,1,0,1,1,1,0,0,1,0,0,positive
381,1,1,1,1,0,1,1,1,1,0,1,1,0,1,0,0,positive
394,1,1,0,1,1,1,1,1,1,1,1,0,1,0,0,1,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,negative
895,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,negative
61,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,negative
1008,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,positive


In [446]:
lazy_fca_alg0("auitsm_df_train", "auitsm_df_test")


True Positive: 360
True Negative: 0 
False Positive: 167
False Negative: 0
True Positive Rate: 1.0
True Negative Rate: 0.0
Negative Predictive Value: nan
False Positive Rate: 1.0
False Discovery Rate: 0.31688804554079697
Accuracy: 0.683111954459203
Precision: 0.683111954459203
Recall: 1.0


<ipython-input-442-1709ed341262>:72: RuntimeWarning: invalid value encountered in true_divide
  NPV = float(TN) / (TN + FN)


In [448]:
lazy_fca_alg1("auitsm_df_train", "auitsm_df_test")


True Positive: 290
True Negative: 167 
False Positive: 0
False Negative: 70
True Positive Rate: 0.8055555555555556
True Negative Rate: 1.0
Negative Predictive Value: 0.7046413502109705
False Positive Rate: 0.0
False Discovery Rate: 0.0
Accuracy: 0.8671726755218216
Precision: 1.0
Recall: 0.8055555555555556


In [449]:
lazy_fca_alg2("auitsm_df_train", "auitsm_df_test")


True Positive: 86
True Negative: 167 
False Positive: 0
False Negative: 274
True Positive Rate: 0.2388888888888889
True Negative Rate: 1.0
Negative Predictive Value: 0.3786848072562358
False Positive Rate: 0.0
False Discovery Rate: 0.0
Accuracy: 0.48007590132827327
Precision: 1.0
Recall: 0.2388888888888889


In [466]:
Class = {'positive': 1,'negative': 0}
auitsm_df_train.Class = [Class[item] for item in auitsm_df_train.Class] 

Class = {'positive': 1,'negative': 0}
auitsm_df_test.Class = [Class[item] for item in auitsm_df_test.Class] 

/Users/edgarzakharyan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [467]:
from sklearn.neighbors import KNeighborsClassifier
def knn_alg(df_train, df_test):
    
    clf = KNeighborsClassifier(n_neighbors=5)
    clf = clf.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1])
     
    df_train = df_train.to_numpy()
    df_test = df_test.to_numpy()
    
# our metrics
    tp = 0
    fp = 0    
    tn = 0
    fn = 0
    
    tp_list = []
    fp_list = []
    tn_list = []
    fn_list = []
    


    
# spliting data into two parts: C+ - context with "+" examples, C- - context with "-" examples:

    plus_context_train = df_train[df_train[:, -1] == 1][:, :-1] 
    minus_context_train = df_train[df_train[:, -1] == 0][:, :-1]
    plus_context_test = df_test[df_test[:, -1] == 1][:, :-1] 
    minus_context_test = df_test[df_test[:, -1] == 0][:, :-1]
    
    TP= int(clf.predict(plus_context_test).sum())
    FN=(1 - clf.predict(plus_context_test)).sum()
    TN=(1 - clf.predict(minus_context_test)).sum()
    FP= int(clf.predict(minus_context_test).sum())
    
    print (
        "True positive:", TP ,"\n" 
        "True Negative:", TN,"\n"
        "False Positive:", FP,"\n"
        "False Negative:", FN,"\n"
        "True Positive Rate:", TP/(TP + FN),"\n"
        "True Negative Rate:", TN/(TN + FP),"\n"
        "Negative Predictive Value:", TN/(TN + FN),"\n"
        "False Positive Rate:",FP/(FP + TN),"\n"
        "False Discovery Rate:",FP/(TP + FP),"\n"
        "Accuracy:" ,(TP+TN)/(TP + FP + TN + FN),"\n"
        "Precision:" ,TP/(TP + FP),"\n"
        "Recall:" ,TP/(TP + FN))
    
    

In [468]:
knn_alg(auitsm_df_train, auitsm_df_test)

True positive: 338 
True Negative: 161 
False Positive: 6 
False Negative: 22 
True Positive Rate: 0.9388888888888889 
True Negative Rate: 0.9640718562874252 
Negative Predictive Value: 0.8797814207650273 
False Positive Rate: 0.03592814371257485 
False Discovery Rate: 0.01744186046511628 
Accuracy: 0.9468690702087287 
Precision: 0.9825581395348837 
Recall: 0.9388888888888889


In [470]:
log_reg(auitsm_df_train, auitsm_df_test)

True positive: 360 
True Negative: 167 
False Positive: 0 
False Negative: 0 
True Positive Rate: 1.0 
True Negative Rate: 1.0 
Negative Predictive Value: 1.0 
False Positive Rate: 0.0 
False Discovery Rate: 0.0 
Accuracy: 1.0 
Precision: 1.0 
Recall: 1.0
